# Génération des codes et merge avec la BDD existante

## Traitements


1. Chargement de la bdd existante et de la nouvelle généré à l'étape 1 (au même format)
2. Merge des données sur matricule, nom, prénom, date de naissance du bénéficiaire (INNER)
3. On garde les nouvelles données pour le résultat du merge
3. Ajout des nouvelles données qui n'étaient pas présentes
4. Génération des codes manquants pour les nouvelles lignes de bénéficiaires
5. Output to csv


## Notes


Création des bénéficiaires manuellement à partir du 27 Juin

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import numpy as np

load_dotenv()

existing_db_export_filepath = os.environ['EXISTING_DB']
new_db_export_filepath = os.environ['DB_EXPORT']
concatenated_db_filepath = os.environ['DB_CONCATENATED']
final_merged_with_ids = os.environ['DB_MERGED_WITH_ID']
final_merged_without_ids = os.environ['DB_MERGED_WITHOUT_ID']

MEMORY_OPTIMIZATION = True

In [ ]:
column_type = {
  'qpv': 'boolean',
  'a_valider': 'boolean',
  'zrr': 'boolean',
}

df_existing_db = pd.read_csv(existing_db_export_filepath, sep=',',  dtype=column_type)

In [ ]:
# Convert to date for comparison, timestamp is added back later down the line
df_existing_db['date_naissance'] = pd.to_datetime(df_existing_db['date_naissance']).dt.date

In [ ]:
# Drop the beneficiaires that have been manually added and need verifications internally to avoid injection conflict
df_existing_db = df_existing_db[df_existing_db['a_valider'] != True]

In [ ]:
df_json_allocataire = pd.json_normalize(df_existing_db['allocataire'].apply(json.loads))
df_json_allocataire = df_json_allocataire.add_prefix('allocataire-')

In [ ]:
df_existing_db.index = pd.RangeIndex(start=0, stop=len(df_existing_db), step=1)
df_existing_db_unwrapped_alloc = pd.merge(df_existing_db, df_json_allocataire, left_index=True, right_index=True)
df_existing_db_unwrapped_alloc = df_existing_db_unwrapped_alloc.drop(columns=['allocataire'])

In [ ]:
if MEMORY_OPTIMIZATION: 
    del df_existing_db
    del df_json_allocataire 

In [ ]:
# Data casting and formating
df_existing_db_unwrapped_alloc['allocataire-matricule'] = df_existing_db_unwrapped_alloc['allocataire-matricule'].astype(str)
df_existing_db_unwrapped_alloc['allocataire-nom'] = df_existing_db_unwrapped_alloc['allocataire-nom'].str.upper()
df_existing_db_unwrapped_alloc['allocataire-prenom'] = df_existing_db_unwrapped_alloc['allocataire-prenom'].str.upper()

# cleaning (telephone)
df_existing_db_unwrapped_alloc['allocataire-telephone'] = df_existing_db_unwrapped_alloc['allocataire-telephone'].replace('0', '')
mask_tel_not_null = df_existing_db_unwrapped_alloc['allocataire-telephone'].notna()
mask_no_zero_phone_number = ~df_existing_db_unwrapped_alloc.loc[mask_tel_not_null, 'allocataire-telephone'].str.startswith('0')
mask_9_char_phone = df_existing_db_unwrapped_alloc.loc[mask_tel_not_null, 'allocataire-telephone'].str.len() == 9
df_existing_db_unwrapped_alloc.loc[mask_tel_not_null & mask_no_zero_phone_number & mask_9_char_phone, 'allocataire-telephone'] = '0' + df_existing_db_unwrapped_alloc['allocataire-telephone']

# replace blank string with nan
df_existing_db_unwrapped_alloc = df_existing_db_unwrapped_alloc.replace(r'', np.NaN)
df_existing_db_unwrapped_alloc = df_existing_db_unwrapped_alloc.replace(r'0', np.NaN)
df_existing_db_unwrapped_alloc = df_existing_db_unwrapped_alloc.replace(r'00000', np.NaN)

# lower case on email
df_existing_db_unwrapped_alloc['allocataire-courriel'] = df_existing_db_unwrapped_alloc['allocataire-courriel'].str.lower()

# check all columns exists, create them otherwise (happens if we act on CNOUS only data)
for column_name in ['allocataire-date_naissance', 'allocataire-pays_naissance',
       'allocataire-commune_naissance', 'allocataire-code_iso_pays_naissance',
       'allocataire-code_insee_commune_naissance']:
    if column_name not in df_existing_db_unwrapped_alloc.columns:
        df_existing_db_unwrapped_alloc[column_name] = np.NaN


# remove duplicate rows (use all relevant columns meaning all minus id_psp, created_at, update_at)
## This implies :
### - Remove these rows from merging
### - No update on these rows, we just keep them in DB
### - No email communication on these row => We don't want the same beneficiary to receive 2 differents code
df_existing_unwrapped_no_duplicate = df_existing_db_unwrapped_alloc.drop_duplicates(subset=[
  'nom', 
  'prenom', 
  'genre',
  'organisme',
  'situation', 
  'allocataire-qualite',
  'allocataire-matricule',
  'allocataire-code_organisme',
  'allocataire-telephone',
  'allocataire-nom',
  'allocataire-prenom',
  'allocataire-date_naissance',
  'allocataire-courriel',
  'allocataire-code_insee_commune_naissance',
  'allocataire-commune_naissance',
  'allocataire-code_iso_pays_naissance',
  'allocataire-pays_naissance'
])

print(f"{len(df_existing_db_unwrapped_alloc) - len(df_existing_unwrapped_no_duplicate)} duplicate rows where removed based on all columns")

In [ ]:
# release
if MEMORY_OPTIMIZATION: 
    del df_existing_db_unwrapped_alloc
    del mask_9_char_phone
    del mask_no_zero_phone_number
    del mask_tel_not_null

In [ ]:
# Loading new data
df_new_db = pd.read_csv(new_db_export_filepath, index_col=0, sep=',')

In [ ]:
# Convert to date for comparison, timestamp is added back later down the line
df_new_db['date_naissance'] = pd.to_datetime(df_new_db['date_naissance']).dt.date

In [ ]:
# json unwrap allocatire
df_json_allocataire_new = pd.json_normalize(df_new_db['allocataire'].apply(json.loads))
df_json_allocataire_new = df_json_allocataire_new.add_prefix('allocataire-')

df_new_db.index = pd.RangeIndex(start=0, stop=len(df_new_db), step=1)
df_new_db_unwrapped_alloc = pd.merge(df_new_db, df_json_allocataire_new, left_index=True, right_index=True)

df_new_db_unwrapped_alloc = df_new_db_unwrapped_alloc.drop(columns=['allocataire'])

In [ ]:
if MEMORY_OPTIMIZATION: 
    del df_new_db
    del df_json_allocataire_new

In [ ]:
# Data casting and formating
df_new_db_unwrapped_alloc['nom'] = df_new_db_unwrapped_alloc['nom'].str.upper()
df_new_db_unwrapped_alloc['prenom'] = df_new_db_unwrapped_alloc['prenom'].str.upper()

mask_matricule_not_null = df_new_db_unwrapped_alloc['allocataire-matricule'].notna()
df_new_db_unwrapped_alloc['allocataire-matricule'] = df_new_db_unwrapped_alloc.loc[mask_matricule_not_null, 'allocataire-matricule'].astype(str)

df_new_db_unwrapped_alloc['allocataire-code_organisme'] = df_new_db_unwrapped_alloc['allocataire-code_organisme'].fillna(0)
df_new_db_unwrapped_alloc['allocataire-code_organisme'] = df_new_db_unwrapped_alloc['allocataire-code_organisme'].astype(int)

# replace blank string with nan
df_new_db_unwrapped_alloc = df_new_db_unwrapped_alloc.replace(r'', np.NaN)

In [ ]:
if MEMORY_OPTIMIZATION: 
    del mask_matricule_not_null

In [ ]:
# backward and forward fill on all columns group to ease merging
def forward_and_backward_fill(group):
    return group.ffill().bfill().iloc[-1]

df_subset_bf_filled = df_existing_unwrapped_no_duplicate.groupby(['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre']) \
                                                 .filter(lambda x: len(x) > 1) \
                                                 .groupby(['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre']) \
                                                 .apply(lambda group: group.ffill().bfill()) \
                                                 .reset_index(drop=True)


In [ ]:
# Re-apply remove duplicate (ignore code and adresse)
df_subset_duplicates_merged = df_subset_bf_filled.drop_duplicates(subset=[
  'nom',
  'prenom',
  'genre',
  'organisme',
  'situation', 
  'allocataire-qualite',
  'allocataire-matricule',
  'allocataire-code_organisme',
  'allocataire-telephone',
  'allocataire-nom',
  'allocataire-prenom',
  'allocataire-date_naissance',
  'allocataire-courriel',
  'allocataire-code_insee_commune_naissance',
  'allocataire-commune_naissance',
  'allocataire-code_iso_pays_naissance',
  'allocataire-pays_naissance'
])

print(f"{len(df_subset_bf_filled) - len(df_subset_duplicates_merged)} duplicate rows were removed based on all columns after forward and backward fills")


In [ ]:
if MEMORY_OPTIMIZATION: 
    del df_subset_bf_filled

In [ ]:
# keep latest update date on existing
df_subset_duplicated_final = df_subset_duplicates_merged.sort_values('updated_at').drop_duplicates(subset=['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre'], keep='last')

print(f"{len(df_subset_duplicates_merged) - len(df_subset_duplicated_final)} duplicate rows where removed based on nom, prenom, matricule, date_naissance and genre")


In [ ]:
if MEMORY_OPTIMIZATION: 
    del df_subset_duplicates_merged

In [ ]:
# retrieve other part
duplicates_from_existing = df_existing_unwrapped_no_duplicate.duplicated(subset=['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre'], keep=False)

# # Keep rows that are not duplicated
df_subset_single = df_existing_unwrapped_no_duplicate[~duplicates_from_existing]

print(f"{len(df_subset_single)} rows are found not duplicated based on 'nom', 'prenom', 'matricule', 'date_naissance' and 'genre'")

In [ ]:
# merging both on existing datas
df_exi_unwrapped_no_duplicate = pd.concat([df_subset_single, df_subset_duplicated_final], axis=0)

In [ ]:
if MEMORY_OPTIMIZATION: 
    del duplicates_from_existing
    del df_subset_single
    del df_subset_duplicated_final

In [ ]:
# we want to ensure no duplicate on existing beneficiary on merge
# Identify duplicates based on the specified columns
duplicates_from_existing = df_exi_unwrapped_no_duplicate.duplicated(subset=['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre'], keep=False)

# Filter the dataframe to get only duplicate rows
df_duplicates_from_existing = df_exi_unwrapped_no_duplicate[duplicates_from_existing]

# Ensure there are no duplicates in the original dataframe
assert len(df_duplicates_from_existing) == 0

In [ ]:
# We want to ensure no duplicate on new
# Identify duplicates based on the specified columns
duplicates_from_new = df_new_db_unwrapped_alloc.duplicated(subset=['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre'], keep=False)

# Filter the dataframe to get only duplicate rows
df_duplicates_from_new = df_new_db_unwrapped_alloc[duplicates_from_new]

# Ensure there are no duplicates in the original dataframe
assert len(df_duplicates_from_existing) == 0

In [ ]:
if MEMORY_OPTIMIZATION: 
    del df_duplicates_from_existing
    del df_duplicates_from_new
    del duplicates_from_existing
    del duplicates_from_new

In [ ]:
# add existing id_psp code and created_at to new data when possible
df_new_unwrapped_no_duplicate_with_id = df_new_db_unwrapped_alloc.merge(\
            df_exi_unwrapped_no_duplicate[['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre', 'id_psp', 'created_at', 'id', 'qpv', 'zrr', 'a_valider']], \
            on=['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre'], \
            suffixes=(None,'_new'), \
            how='left')

mask_has_code = ~df_new_unwrapped_no_duplicate_with_id['id_psp'].isna()

print(f"{len(df_new_unwrapped_no_duplicate_with_id[mask_has_code])} rows already have a code, created_at is copied on them")

In [ ]:
if MEMORY_OPTIMIZATION: 
    del mask_has_code
    del df_new_db_unwrapped_alloc

In [ ]:
# on concat pour former en théorie des paires de doublons dans l'optique de supprimer les duplicats entre les nouvelles et les anciennes données
df_concatenated = pd.concat([df_new_unwrapped_no_duplicate_with_id, df_exi_unwrapped_no_duplicate], axis=0).reset_index(drop=True)

In [ ]:
# map 'jeune' to 'Jeune'
df_concatenated['situation'] = df_concatenated['situation'].replace('jeune', 'Jeune')

In [ ]:
if MEMORY_OPTIMIZATION: 
    del df_new_unwrapped_no_duplicate_with_id
    del df_exi_unwrapped_no_duplicate

In [ ]:
# drop duplicate, new data prevails
df_concatenated_no_duplicates = df_concatenated.drop_duplicates(subset=['nom', 'prenom', 'allocataire-matricule', 'date_naissance', 'genre'], keep='first')

print(f"{len(df_concatenated) - len(df_concatenated_no_duplicates)} duplicated rows deleted based on 'nom', 'prenom', 'matricule', 'date_naissance' and 'genre', latest data are kept")
# TODO: (optional) we could add a step here to ensure data from existing are kept if no value are present in the datas

In [ ]:
# Retrieve those without code
mask_no_code = df_concatenated_no_duplicates['id_psp'].isna()
df_no_code = df_concatenated_no_duplicates.loc[mask_no_code]

print(f"{len(df_no_code)} rows without code")
print(f"{len(df_concatenated_no_duplicates[~mask_no_code])} rows with code")

In [ ]:
# generate new code ensuring no duplicates with existings
import random
import string
import datetime

current_date = datetime.datetime.now()
current_year = str(current_date.year)[-2:]

def get_characters_set(size = 4):
    return ''.join(random.choices([c for c in string.ascii_uppercase if c not in 'OI'], k=size))
    
def generate_code():
    return f"{current_year}-{get_characters_set(4)}-{get_characters_set(4)}"

# init set of codes with existing
unique_codes = set(df_concatenated_no_duplicates[~mask_no_code]['id_psp'])

# init current_code count
current_codes_count = len(unique_codes)
while len(unique_codes) < current_codes_count + len(df_no_code):
    code = generate_code()
    unique_codes.add(code)

# only retrieve newly created codes
new_codes = list(unique_codes.difference(df_concatenated_no_duplicates['id_psp']))
df_new_codes = pd.DataFrame({'id_psp': new_codes})

print(f"{len(df_new_codes)} generated codes")

In [ ]:
# merge with no_code dataframe
df_final_with_new_code = df_no_code.reset_index(drop=True).combine_first(df_new_codes.reset_index(drop=True))

In [ ]:
if MEMORY_OPTIMIZATION:
    del df_no_code
    del df_new_codes

In [ ]:
# concat with already existing codes dataframe 
mask_has_code = ~df_concatenated_no_duplicates['id_psp'].isna()
df_has_code = df_concatenated_no_duplicates[mask_has_code]
df_final_with_code = pd.concat([df_has_code, df_final_with_new_code], axis=0)

In [ ]:
if MEMORY_OPTIMIZATION:
    del df_has_code
    del df_concatenated_no_duplicates

In [ ]:
# ensure everyone has a code
mask_no_code = df_final_with_code['id_psp'].isna()
df_still_no_code = df_final_with_code[mask_no_code]
assert len(df_still_no_code) == 0

In [ ]:
if MEMORY_OPTIMIZATION:
    del df_still_no_code
    del mask_no_code

In [ ]:
df_final_with_code = df_final_with_code.reset_index(drop=True)

In [ ]:
import pytz

# add created_at for new rows
# using time zone for created_at to be iso existings
tz = pytz.timezone('Europe/Paris')
now = datetime.datetime.now()
now_tz = tz.localize(now)

mask_no_created_at = df_final_with_code['created_at'].isna()

df_final_with_code.loc[mask_no_created_at, 'created_at'] = now_tz
df_final_with_code[['zrr', 'qpv', 'a_valider']].fillna(value=False, inplace=True)

In [ ]:
# map to json values for target DB model 
def to_json_allocataire_without_null(row):
    allocataire_mapping = {
        'qualite': row['allocataire-qualite'],
        'matricule': row['allocataire-matricule'],
        'code_organisme': row['allocataire-code_organisme'],
        'telephone': row['allocataire-telephone'],
        'nom': row['allocataire-nom'],
        'prenom': row['allocataire-prenom'],
        'date_naissance': row['allocataire-date_naissance'],
        'courriel': row['allocataire-courriel'],
        'code_insee_commune_naissance': row['allocataire-code_insee_commune_naissance'],
        'commune_naissance': row['allocataire-commune_naissance'],
        'code_iso_pays_naissance': row['allocataire-code_iso_pays_naissance'],
        'pays_naissance': row['allocataire-pays_naissance']
    }
    filtered_nan_allocataire = {k: v for k, v in allocataire_mapping.items() if pd.notnull(v)}
    return json.dumps(filtered_nan_allocataire, ensure_ascii=False)

df_final_with_code['allocataire'] = df_final_with_code.apply(to_json_allocataire_without_null, axis=1)

In [ ]:
df_final_with_code.loc[df_final_with_code['refuser'].isna(), 'refuser'] = False

In [ ]:
df_final_with_code = df_final_with_code.drop(columns=[
  'allocataire-qualite',
  'allocataire-matricule',
  'allocataire-code_organisme',
  'allocataire-nom',
  'allocataire-prenom',
  'allocataire-telephone',
  'allocataire-date_naissance',
  'allocataire-courriel',
  'allocataire-code_insee_commune_naissance',
  'allocataire-commune_naissance',
  'allocataire-code_iso_pays_naissance',
  'allocataire-pays_naissance',
  'allocataire-departement_naissance'
])

In [ ]:
# Add back timestamp and the 4 hours to be iso with the existing database
df_final_with_code['date_naissance'] = pd.to_datetime(df_final_with_code['date_naissance']).dt.floor('D') + pd.DateOffset(hours=4)

In [ ]:
# Take existing
df_final_existing_with_codes = df_final_with_code[~df_final_with_code['id'].isna()]
len(df_final_existing_with_codes)

In [ ]:
# Take new
df_final_existing_without_codes = df_final_with_code[df_final_with_code['id'].isna()]
len(df_final_existing_without_codes)

In [ ]:
df_final_existing_with_codes.to_csv(final_merged_with_ids, index=False)
df_final_existing_without_codes.to_csv(final_merged_without_ids, index=False)